In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-2.0_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-2.0_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "59.159224555735065,695.847422157399,538.2046526655897,105"
                              ],
                          columns=[
                              "83.68865912762521,195.27353796445882,226.0555735056543,252.98985460420036,278.962197092084,303.010662358643,334.75463651050086,364.57473344103397,383.81350565428113,414.59554119547664,446.33951534733444,480.9693053311794,501.1700161550889"
                              ],
                              row_tol=12,
                          split_text=True,
                          strip_text='\n',
                          pages="all"
                         )

In [6]:
tables

<TableList n=2>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [ ]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
column_header = [
    ['Below 18 years','18 yrs. – Below 30 years','30 yrs. – Below 45 years'],
    ['45 yrs. – Below 60 years','60 years & above','Total']
]
len(column_header)

2

In [10]:
def set_Column(df, title):
    column_list = [
        ("Sl. No.",''),
        ("Cause",''), 
        (title[0],'Male'),
        (title[0],'Female'),
        (title[0],'Transgender'),
        (title[0],'Total'),
        (title[1],'Male'),
        (title[1],'Female'),
        (title[1],'Transgender'),
        (title[1],'Total'),
        (title[2],'Male'),
        (title[2],'Female'),
        (title[2],'Transgender'),
        (title[2],'Total')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [11]:
for i in range(len(tables)):
    set_Column(tables[i].df, column_header[i])
    tables[i].df.set_index(["Sl. No.","Cause"],inplace=True)

In [12]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [13]:
#Copying Table 1 index into Table 2 for uniformity when concatenating
tables[1].df.index = tables[0].df.index.copy()

In [14]:
# Convert List of tables to list of DataFrames
DataFrames = [each.df.copy() for each in tables]
# Concat all the tables one after the other, as they all have the same indexes.
final = pd.concat(DataFrames,axis=1).reset_index()

In [15]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(DF, strIndex, stpIndex):
    df = DF.iloc[strIndex:stpIndex+1].copy()
    for index, each in df.iterrows():
        space = each['Cause'][0].find(' ')
        each['Sl. No.'] = each['Cause'][0][:space]
        each['Cause'] = each['Cause'][0][space:]
    DF.iloc[strIndex:stpIndex+1] = df

In [16]:
split_by_first_space(final,2,6)
split_by_first_space(final,11,15)

In [17]:
# Reset Index before saving file, for better formatting in RAW CSV
final.to_csv("Age and Gender – wise Distribution of Suicides during 2019 (Cause-wise).csv",index=False)